In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import metrics

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import tensorflow as tf
import os
import time
import gc
import re
import random
from tqdm import tqdm
tqdm.pandas()
from nltk.stem.wordnet import WordNetLemmatizer

Using TensorFlow backend.


In [2]:
#设置随机种子保证可重复性
def seed_everything(seed=1234):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything()

In [3]:
#读入数据，预处理，包括改为小写和替换数字和url链接
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")

train["question_text"] = train["question_text"].str.lower()
test["question_text"] = test["question_text"].str.lower()

def clean_tag(text):
    if '[math]' in text:
        text = re.sub('\[math\].*?math\]', '[formula]', text)
    if 'http' in text or 'www' in text:
        text = re.sub('(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+', '[url]', text)
    return text
train["question_text"] = train["question_text"].apply(lambda x: clean_tag(x))
test["question_text"] = test["question_text"].apply(lambda x: clean_tag(x))

In [4]:
puncts=[',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', 
        '•', '~', '@', '£', '·', '_', '{', '}', '©', '^', '®', '`', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 
        '█', '…', '“', '★', '”', '–', '●', '►', '−', '¢', '¬', '░', '¡', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', 
        '—', '‹', '─', '▒', '：', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', '¯', '♦', '¤', '▲', '¸', '⋅', '‘', '∞', 
        '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '・', '╦', '╣', '╔', '╗', '▬', '❤', '≤', '‡', '√', '◄', '━', 
        '⇒', '▶', '≥', '╝', '♡', '◊', '。', '✈', '≡', '☺', '✔', '↵', '≈', '✓', '♣', '☎', '℃', '◦', '└', '‟', '～', '！', '○', 
        '◆', '№', '♠', '▌', '✿', '▸', '⁄', '□', '❖', '✦', '．', '÷', '｜', '┃', '／', '￥', '╠', '↩', '✭', '▐', '☼', '☻', '┐', 
        '├', '«', '∼', '┌', '℉', '☮', '฿', '≦', '♬', '✧', '〉', '－', '⌂', '✖', '･', '◕', '※', '‖', '◀', '‰', '\x97', '↺', 
        '∆', '┘', '┬', '╬', '،', '⌘', '⊂', '＞', '〈', '⎙', '？', '☠', '⇐', '▫', '∗', '∈', '≠', '♀', '♔', '˚', '℗', '┗', '＊', 
        '┼', '❀', '＆', '∩', '♂', '‿', '∑', '‣', '➜', '┛', '⇓', '☯', '⊖', '☀', '┳', '；', '∇', '⇑', '✰', '◇', '♯', '☞', '´', 
        '↔', '┏', '｡', '◘', '∂', '✌', '♭', '┣', '┴', '┓', '✨', '\xa0', '˜', '❥', '┫', '℠', '✒', '［', '∫', '\x93', '≧', '］', 
        '\x94', '∀', '♛', '\x96', '∨', '◎', '↻', '⇩', '＜', '≫', '✩', '✪', '♕', '؟', '₤', '☛', '╮', '␊', '＋', '┈', '％', 
        '╋', '▽', '⇨', '┻', '⊗', '￡', '।', '▂', '✯', '▇', '＿', '➤', '✞', '＝', '▷', '△', '◙', '▅', '✝', '∧', '␉', '☭', 
        '┊', '╯', '☾', '➔', '∴', '\x92', '▃', '↳', '＾', '׳', '➢', '╭', '➡', '＠', '⊙', '☢', '˝', '∏', '„', '∥', '❝', '☐', 
        '▆', '╱', '⋙', '๏', '☁', '⇔', '▔', '\x91', '➚', '◡', '╰', '\x85', '♢', '˙', '۞', '✘', '✮', '☑', '⋆', 'ⓘ', '❒', 
        '☣', '✉', '⌊', '➠', '∣', '❑', '◢', 'ⓒ', '\x80', '〒', '∕', '▮', '⦿', '✫', '✚', '⋯', '♩', '☂', '❞', '‗', '܂', '☜', 
        '‾', '✜', '╲', '∘', '⟩', '＼', '⟨', '·', '✗', '♚', '∅', 'ⓔ', '◣', '͡', '‛', '❦', '◠', '✄', '❄', '∃', '␣', '≪', '｢', 
        '≅', '◯', '☽', '∎', '｣', '❧', '̅', 'ⓐ', '↘', '⚓', '▣', '˘', '∪', '⇢', '✍', '⊥', '＃', '⎯', '↠', '۩', '☰', '◥', 
        '⊆', '✽', '⚡', '↪', '❁', '☹', '◼', '☃', '◤', '❏', 'ⓢ', '⊱', '➝', '̣', '✡', '∠', '｀', '▴', '┤', '∝', '♏', 'ⓐ', 
        '✎', ';', '␤', '＇', '❣', '✂', '✤', 'ⓞ', '☪', '✴', '⌒', '˛', '♒', '＄', '✶', '▻', 'ⓔ', '◌', '◈', '❚', '❂', '￦', 
        '◉', '╜', '̃', '✱', '╖', '❉', 'ⓡ', '↗', 'ⓣ', '♻', '➽', '׀', '✲', '✬', '☉', '▉', '≒', '☥', '⌐', '♨', '✕', 'ⓝ', 
        '⊰', '❘', '＂', '⇧', '̵', '➪', '▁', '▏', '⊃', 'ⓛ', '‚', '♰', '́', '✏', '⏑', '̶', 'ⓢ', '⩾', '￠', '❍', '≃', '⋰', '♋', 
        '､', '̂', '❋', '✳', 'ⓤ', '╤', '▕', '⌣', '✸', '℮', '⁺', '▨', '╨', 'ⓥ', '♈', '❃', '☝', '✻', '⊇', '≻', '♘', '♞', 
        '◂', '✟', '⌠', '✠', '☚', '✥', '❊', 'ⓒ', '⌈', '❅', 'ⓡ', '♧', 'ⓞ', '▭', '❱', 'ⓣ', '∟', '☕', '♺', '∵', '⍝', 'ⓑ', 
        '✵', '✣', '٭', '♆', 'ⓘ', '∶', '⚜', '◞', '்', '✹', '➥', '↕', '̳', '∷', '✋', '➧', '∋', '̿', 'ͧ', '┅', '⥤', '⬆', '⋱', 
        '☄', '↖', '⋮', '۔', '♌', 'ⓛ', '╕', '♓', '❯', '♍', '▋', '✺', '⭐', '✾', '♊', '➣', '▿', 'ⓑ', '♉', '⏠', '◾', '▹', 
        '⩽', '↦', '╥', '⍵', '⌋', '։', '➨', '∮', '⇥', 'ⓗ', 'ⓓ', '⁻', '⎝', '⌥', '⌉', '◔', '◑', '✼', '♎', '♐', '╪', '⊚', 
        '☒', '⇤', 'ⓜ', '⎠', '◐', '⚠', '╞', '◗', '⎕', 'ⓨ', '☟', 'ⓟ', '♟', '❈', '↬', 'ⓓ', '◻', '♮', '❙', '♤', '∉', '؛', 
        '⁂', 'ⓝ', '־', '♑', '╫', '╓', '╳', '⬅', '☔', '☸', '┄', '╧', '׃', '⎢', '❆', '⋄', '⚫', '̏', '☏', '➞', '͂', '␙', 
        'ⓤ', '◟', '̊', '⚐', '✙', '↙', '̾', '℘', '✷', '⍺', '❌', '⊢', '▵', '✅', 'ⓖ', '☨', '▰', '╡', 'ⓜ', '☤', '∽', '╘', 
        '˹', '↨', '♙', '⬇', '♱', '⌡', '⠀', '╛', '❕', '┉', 'ⓟ', '̀', '♖', 'ⓚ', '┆', '⎜', '◜', '⚾', '⤴', '✇', '╟', '⎛', 
        '☩', '➲', '➟', 'ⓥ', 'ⓗ', '⏝', '◃', '╢', '↯', '✆', '˃', '⍴', '❇', '⚽', '╒', '̸', '♜', '☓', '➳', '⇄', '☬', '⚑', 
        '✐', '⌃', '◅', '▢', '❐', '∊', '☈', '॥', '⎮', '▩', 'ு', '⊹', '‵', '␔', '☊', '➸', '̌', '☿', '⇉', '⊳', '╙', 'ⓦ', 
        '⇣', '｛', '̄', '↝', '⎟', '▍', '❗', '״', '΄', '▞', '◁', '⛄', '⇝', '⎪', '♁', '⇠', '☇', '✊', 'ி', '｝', '⭕', '➘', 
        '⁀', '☙', '❛', '❓', '⟲', '⇀', '≲', 'ⓕ', '⎥', '\u06dd', 'ͤ', '₋', '̱', '̎', '♝', '≳', '▙', '➭', '܀', 'ⓖ', '⇛', '▊', 
        '⇗', '̷', '⇱', '℅', 'ⓧ', '⚛', '̐', '̕', '⇌', '␀', '≌', 'ⓦ', '⊤', '̓', '☦', 'ⓕ', '▜', '➙', 'ⓨ', '⌨', '◮', '☷', 
        '◍', 'ⓚ', '≔', '⏩', '⍳', '℞', '┋', '˻', '▚', '≺', 'ْ', '▟', '➻', '̪', '⏪', '̉', '⎞', '┇', '⍟', '⇪', '▎', '⇦', '␝', 
        '⤷', '≖', '⟶', '♗', '̴', '♄', 'ͨ', '̈', '❜', '̡', '▛', '✁', '➩', 'ா', '˂', '↥', '⏎', '⎷', '̲', '➖', '↲', '⩵', '̗', '❢', 
        '≎', '⚔', '⇇', '̑', '⊿', '̖', '☍', '➹', '⥊', '⁁', '✢']

In [5]:
#替换特殊的符号
def clean_punct(x):
    for punct in puncts:
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x
train["question_text"] = train["question_text"].apply(lambda x: clean_punct(x))
test["question_text"] = test["question_text"].apply(lambda x: clean_punct(x))

In [6]:
#超参数
embed_size = 300 # how big is each word vector
max_features = 200000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 72 # max number of words in a question to use #99.99%

## fill up the missing values
X = train["question_text"].fillna("_####_").values
X_test = test["question_text"].fillna("_####_").values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features, filters='')
tokenizer.fit_on_texts(list(X)+list(X_test))

X = tokenizer.texts_to_sequences(X)
X_test = tokenizer.texts_to_sequences(X_test)

## Pad the sentences 
X = pad_sequences(X, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

## Get the target values
Y = train['target'].values

sub = test[['qid']]
del train, test

In [7]:
#导入词向量，并作一些简单处理使得单词更加全面
word_index = tokenizer.word_index
max_features = len(word_index)+1
def load_glove(word_index):
    EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) 
                            for o in open(EMBEDDING_FILE) 
                            if o.split(" ")[0] in word_index or o.split(" ")[0].lower() in word_index)

    emb_mean, emb_std = -0.005838499, 0.48782197
    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:                            #原单词
            embedding_matrix[i] = embedding_vector
        elif embeddings_index.get(word.capitalize()) is not None:    #首字母大写，其他小写
            embedding_matrix[i] = embeddings_index.get(word.capitalize())
        elif embeddings_index.get(word.upper()) is not None:         #小写单词
            embedding_matrix[i] = embeddings_index.get(word.upper())
    del embeddings_index
    gc.collect()        
    return embedding_matrix 

def load_para(word_index):
    EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) 
                            for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') 
                            if len(o)>100 and (o.split(" ")[0] in word_index or o.split(" ")[0].lower() in word_index))

    emb_mean, emb_std = -0.005838499, 0.48782197
    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: 
            embedding_matrix[i] = embedding_vector
        elif embeddings_index.get(word.capitalize()) is not None:
            embedding_matrix[i] = embeddings_index.get(word.capitalize())
        elif embeddings_index.get(word.upper()) is not None:
            embedding_matrix[i] = embeddings_index.get(word.upper())
        
    del embeddings_index
    gc.collect()
    return embedding_matrix

seed_everything()
embedding_matrix_1 = load_glove(word_index)
embedding_matrix_3 = load_para(word_index)

#词向量按不同权重平均
embedding_matrix = np.mean((1.28*embedding_matrix_1, 0.72*embedding_matrix_3), axis=0)
del embedding_matrix_1, embedding_matrix_3
gc.collect()
np.shape(embedding_matrix)

(223748, 300)

In [8]:
#定义attention层和adamW
class AttentionWeightedAverage(Layer):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for a single timestep.
    """

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.RandomUniform(seed=10000)
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttentionWeightedAverage, self).__init__(** kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self.trainable_weights = [self.W]
        super(AttentionWeightedAverage, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None
class AdamW(Optimizer):
    def __init__(self, lr=0.001, beta_1=0.9, beta_2=0.999, weight_decay=1e-4,  # decoupled weight decay (1/4)
                 epsilon=1e-8, decay=0., **kwargs):
        super(AdamW, self).__init__(**kwargs)
        with K.name_scope(self.__class__.__name__):
            self.iterations = K.variable(0, dtype='int64', name='iterations')
            self.lr = K.variable(lr, name='lr')
            self.beta_1 = K.variable(beta_1, name='beta_1')
            self.beta_2 = K.variable(beta_2, name='beta_2')
            self.decay = K.variable(decay, name='decay')
            self.wd = K.variable(weight_decay, name='weight_decay') # decoupled weight decay (2/4)
        self.epsilon = epsilon
        self.initial_decay = decay

    @interfaces.legacy_get_updates_support
    def get_updates(self, loss, params):
        grads = self.get_gradients(loss, params)
        self.updates = [K.update_add(self.iterations, 1)]
        wd = self.wd # decoupled weight decay (3/4)

        lr = self.lr
        if self.initial_decay > 0:
            lr *= (1. / (1. + self.decay * K.cast(self.iterations,
                                                  K.dtype(self.decay))))

        t = K.cast(self.iterations, K.floatx()) + 1
        lr_t = lr * (K.sqrt(1. - K.pow(self.beta_2, t)) /
                     (1. - K.pow(self.beta_1, t)))

        ms = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        vs = [K.zeros(K.int_shape(p), dtype=K.dtype(p)) for p in params]
        self.weights = [self.iterations] + ms + vs

        for p, g, m, v in zip(params, grads, ms, vs):
            m_t = (self.beta_1 * m) + (1. - self.beta_1) * g
            v_t = (self.beta_2 * v) + (1. - self.beta_2) * K.square(g)
            p_t = p - lr_t * m_t / (K.sqrt(v_t) + self.epsilon) - lr * wd * p # decoupled weight decay (4/4)

            self.updates.append(K.update(m, m_t))
            self.updates.append(K.update(v, v_t))
            new_p = p_t

            # Apply constraints.
            if getattr(p, 'constraint', None) is not None:
                new_p = p.constraint(new_p)

            self.updates.append(K.update(p, new_p))
        return self.updates

    def get_config(self):
        config = {'lr': float(K.get_value(self.lr)),
                  'beta_1': float(K.get_value(self.beta_1)),
                  'beta_2': float(K.get_value(self.beta_2)),
                  'decay': float(K.get_value(self.decay)),
                  'weight_decay': float(K.get_value(self.wd)),
                  'epsilon': self.epsilon}
        base_config = super(AdamW, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [9]:
#定义模型结构1
def LSTM_GRU(spatialdropout=0.20, rnn_units=64, weight_decay=0.07):
    K.clear_session()       
    x_input = Input(shape=(maxlen,))
    
    emb = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False, name='Embedding')(x_input)
    emb = SpatialDropout1D(spatialdropout, seed=1024)(emb)

    rnn1 = Bidirectional(CuDNNLSTM(rnn_units, return_sequences=True, kernel_initializer=glorot_uniform(seed=111100), 
                           recurrent_initializer=Orthogonal(gain=1.0, seed=123000)))(emb)
    rnn2 = Bidirectional(CuDNNGRU(rnn_units, return_sequences=True, kernel_initializer=glorot_uniform(seed=111000), 
                           recurrent_initializer=Orthogonal(gain=1.0, seed=1203000)))(rnn1)

    x = concatenate([rnn1, rnn2])
    x = GlobalMaxPooling1D()(x)  
    x_output = Dense(1, activation='sigmoid', kernel_initializer=glorot_uniform(seed=111100))(x)
    
    model = Model(inputs=x_input, outputs=x_output)
    model.compile(loss='binary_crossentropy', optimizer=AdamW(weight_decay=weight_decay),)
    return model

In [10]:
#定义模型结构2
#epoch=7
def poolRNN(spatialdropout=0.2, gru_units=128, weight_decay=0.04):
    K.clear_session()
    inp = Input(shape=(maxlen,))
    embedding_layer = Embedding(max_features,
                                embed_size,
                                weights=[embedding_matrix],
                                input_length=maxlen,
                                trainable=False)(inp)
    embedding_layer = SpatialDropout1D(spatialdropout, seed=1024)(embedding_layer)

    rnn_1 = Bidirectional(CuDNNGRU(gru_units, return_sequences=True, 
                                   kernel_initializer=glorot_uniform(seed=10000), 
                                   recurrent_initializer=Orthogonal(gain=1.0, seed=123000)))(embedding_layer)

    last = Lambda(lambda t: t[:, -1], name='last')(rnn_1)
    maxpool = GlobalMaxPooling1D()(rnn_1)
    attn = AttentionWeightedAverage()(rnn_1)
    average = GlobalAveragePooling1D()(rnn_1)

    c = concatenate([last, maxpool, attn], axis=1)
    c = Reshape((3, -1))(c)
    c = Lambda(lambda x:K.sum(x, axis=1))(c)
    x = BatchNormalization()(c)
    x = Dense(200, activation='relu', kernel_initializer=glorot_uniform(seed=111000))(x)
    x = Dropout(0.2, seed=1024)(x)
    x = BatchNormalization()(x)
    output_layer = Dense(1, activation="sigmoid", kernel_initializer=glorot_uniform(seed=111000))(x)
    model = Model(inputs=inp, outputs=output_layer)
    model.compile(loss='binary_crossentropy', optimizer=AdamW(weight_decay=weight_decay))
    return model

In [11]:
#定义模型结构3
def BiLSTM_CNN(spatialdropout=0.2, rnn_units=128, filters=[100, 80, 30, 12], weight_decay=0.10):
    K.clear_session()       
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(rate=spatialdropout, seed=10000)(x)
    x = Bidirectional(CuDNNLSTM(rnn_units, return_sequences=True, 
                               kernel_initializer=glorot_uniform(seed=111000), 
                               recurrent_initializer=Orthogonal(gain=1.0, seed=123000)))(x)

    x1 = Conv1D(filters=filters[0], activation='relu', kernel_size=1, 
                padding='same', kernel_initializer=glorot_uniform(seed=110000))(x)
    x2 = Conv1D(filters=filters[1], activation='relu', kernel_size=2, 
                padding='same', kernel_initializer=glorot_uniform(seed=120000))(x)
    x3 = Conv1D(filters=filters[2], activation='relu', kernel_size=3, 
                padding='same', kernel_initializer=glorot_uniform(seed=130000))(x)
    x4 = Conv1D(filters=filters[3], activation='relu', kernel_size=5, 
                padding='same', kernel_initializer=glorot_uniform(seed=140000))(x)

    
    x1 = GlobalMaxPool1D()(x1)
    x2 = GlobalMaxPool1D()(x2)
    x3 = GlobalMaxPool1D()(x3)
    x4 = GlobalMaxPool1D()(x4)

    c = concatenate([x1, x2, x3, x4])
    x = Dense(200, activation='relu', kernel_initializer=glorot_uniform(seed=111000))(c)
    x = Dropout(0.2, seed=10000)(x)
    x = BatchNormalization()(x)
    x = Dense(1, activation="sigmoid", kernel_initializer=glorot_uniform(seed=110000))(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer=AdamW(weight_decay=weight_decay))
    return model

In [12]:
#定义模型结构4
#epoch=7
def singleRNN(spatialdropout=0.20, rnn_units=120, weight_decay=0.08):
    K.clear_session()
    inp = Input(shape=(maxlen,))
    embedding_layer = Embedding(max_features,
                                embed_size,
                                weights=[embedding_matrix],
                                input_length=maxlen,
                                trainable=False)(inp)
    embedding_layer = SpatialDropout1D(spatialdropout, seed=1024)(embedding_layer)

    x = Bidirectional(CuDNNGRU(rnn_units, return_sequences=True, 
                                   kernel_initializer=glorot_uniform(seed=111000), 
                                   recurrent_initializer=Orthogonal(gain=1.0, seed=123000)))(embedding_layer)

    x = AttentionWeightedAverage()(x)
    x = Dense(100, kernel_initializer=glorot_uniform(seed=111000))(x)
    x = Dropout(0.12, seed=1024)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    output_layer = Dense(1, activation="sigmoid", kernel_initializer=glorot_uniform(seed=111000))(x)
    model = Model(inputs=inp, outputs=output_layer)
    model.compile(loss='binary_crossentropy', optimizer=AdamW(weight_decay=weight_decay))
    return model

In [13]:
#定义f1函数，可以得出阀值
def f1_smart(y_true, y_pred):
    args = np.argsort(y_pred)
    tp = y_true.sum()
    fs = (tp - np.cumsum(y_true[args[:-1]])) / np.arange(y_true.shape[0] + tp - 1, tp, -1)
    res_idx = np.argmax(fs)
    return 2 * fs[res_idx], (y_pred[args[res_idx]] + y_pred[args[res_idx + 1]]) / 2

In [14]:
#定义训练。
kfold = StratifiedKFold(n_splits=7, random_state=10, shuffle=True)
bestscore = []
bestloss = []
y_test = np.zeros((X_test.shape[0], ))
oof = np.zeros((X.shape[0], ))
epochs = [8, 8, 7, 6]
val_list = []
for i, (train_index, valid_index) in enumerate(kfold.split(X, Y)):
    val_list += list(valid_index)
    print('FOLD%s'%(i+1))
    X_train, X_val, Y_train, Y_val = X[train_index], X[valid_index], Y[train_index], Y[valid_index]
    filepath="weights_best.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='min')
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0, verbose=0)
    callbacks = [checkpoint, reduce_lr]
    if i == 0:
        model = LSTM_GRU(spatialdropout=0.20, rnn_units=64, weight_decay=0.07)
        print('LSTM_GRU(spatialdropout=0.20, rnn_units=64, weight_decay=0.07)')
    elif i == 1:
        model = poolRNN(spatialdropout=0.2, gru_units=128, weight_decay=0.04)
        print('poolRNN(spatialdropout=0.2, gru_units=128, weight_decay=0.04)')
    elif i == 2:
        model = BiLSTM_CNN(spatialdropout=0.2, rnn_units=128, filters=[100, 90, 30, 12], weight_decay=0.10)
        print('BiLSTM_CNN(spatialdropout=0.2, rnn_units=128, filters=[100, 90, 30, 12], weight_decay=0.10)')
    model.fit(X_train, Y_train, batch_size=512, epochs=epochs[i], 
              validation_data=(X_val, Y_val), verbose=0, callbacks=callbacks, 
              #class_weight={0:1, 1:1.25}
             )
    print("train logloss:%s"%model.history.history['loss'])
    print("val logloss:%s"%model.history.history['val_loss'])
    y_pred = model.predict([X_val], batch_size=1024, verbose=2)
    y_test += np.squeeze(model.predict([X_test], batch_size=1024, verbose=2))/3
    oof[valid_index] = np.squeeze(y_pred)
    f1, threshold = f1_smart(np.squeeze(Y_val), np.squeeze(y_pred))
    print('Optimal F1: {:.4f} at threshold: {:.4f}\n'.format(f1, threshold))
    bestscore.append(threshold)
    if i == 2:break

FOLD1
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
LSTM_GRU(spatialdropout=0.20, rnn_units=64, weight_decay=0.07)
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
train logloss:[0.11759099725726352, 0.10276792913609609, 0.09815277054641339, 0.09491142912159056, 0.09241925636582358, 0.09025414252364734, 0.08845839995601772, 0.08209750478683663]
val logloss:[0.10240384631193913, 0.09824021655529716, 0.09553803843447324, 0.0951386381953506, 0.09458865939379313, 0.09407106672741923, 0.09436700971764865, 0.0930861053193227]
Optimal F1: 0.6972 at threshold: 0.3174

FOLD2
poolRNN(spatialdropout=0.2, gru_units=128, weight_decay=0.04)
train logloss:[0.1378395096501395, 0.10408810704023005, 0.09918862019047445, 0.09501516070182062, 0.09200669502816546, 0.08905362274665601, 0.08123

In [15]:
f1, threshold = f1_smart(np.squeeze(Y[val_list]), np.squeeze(oof[val_list]))
print('Optimal F1: {:.4f} at threshold: {:.4f}'.format(f1, threshold))

Optimal F1: 0.6991 at threshold: 0.3325


In [16]:
y_test = y_test.reshape((-1, 1)) 
pred_test_y = (y_test>threshold).astype(int) 
sub['prediction'] = pred_test_y 
sub.to_csv("submission.csv", index=False)